<a href="https://colab.research.google.com/github/nattaran/health-tequity-case-nasrin/blob/main/Generate_Spanish_Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
 !pip install gtts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.0
    Uninstalling click-8.3.0:
      Successfully uninstalled click-8.3.0


In [7]:
!pip install -q deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.0 MB/s eta 0:00:00


#Text -> Spanish Audio Generator (Clean Folder Each Run)


In [8]:
# ================================================================
# 🗣️  ENGLISH → SPANISH AUDIO GENERATION MODULE (Colab + Drive)
# ================================================================

from gtts import gTTS
from deep_translator import GoogleTranslator
import os, pandas as pd

def generate_spanish_audio_from_english(
    english_questions: list[str],
    output_folder: str,
    prefix: str = "q"
):
    """
    Translates each English question into Spanish, generates new Spanish audio files,
    and saves both 'generated_questions.csv' and 'ground_truth.csv' for later pipeline steps.

    Args:
        english_questions: list of English question strings
        output_folder: path to save generated .wav files
        prefix: filename prefix for generated audio files (default 'q')

    Returns:
        DataFrame containing English text, Spanish translation, and audio filenames
    """

    # ============================================================
    # 🧹 Step 1: Create or clean the output folder
    # ============================================================
    if os.path.exists(output_folder):
        print(f"🧹 Cleaning old files in {output_folder} ...")
        for f in os.listdir(output_folder):
            fp = os.path.join(output_folder, f)
            if os.path.isfile(fp):
                os.remove(fp)
    else:
        os.makedirs(output_folder, exist_ok=True)
        print(f"📁 Created new folder: {output_folder}")

    # ============================================================
    # 🌍 Step 2: Translate English → Spanish and generate audio
    # ============================================================
    translator = GoogleTranslator(source="en", target="es")
    results = []

    for i, question_en in enumerate(english_questions, 1):
        try:
            # Translate English → Spanish
            question_es = translator.translate(question_en)

            # Generate audio (Spanish)
            tts = gTTS(text=question_es, lang="es")
            audio_filename = f"{prefix}{i}_es.wav"
            audio_path = os.path.join(output_folder, audio_filename)
            tts.save(audio_path)

            results.append({
                "question_number": i,
                "english_text": question_en,
                "spanish_text": question_es,
                "audio_file": audio_filename
            })

            print(f"🎧 {audio_filename} generated → {question_es}")

        except Exception as e:
            print(f"⚠️ Error generating audio for question {i}: {e}")

    # ============================================================
    # 🧾 Step 3: Save CSV files
    # ============================================================

    # Summary of generated questions
    summary_csv = os.path.join(output_folder, "generated_questions.csv")
    pd.DataFrame(results).to_csv(summary_csv, index=False, encoding="utf-8-sig")

    # Ground truth file for WER evaluation
    gt_df = pd.DataFrame({
        "audio_file": [r["audio_file"] for r in results],
        "ground_truth": [r["spanish_text"] for r in results]
    })
    gt_csv = os.path.join(output_folder, "ground_truth.csv")
    gt_df.to_csv(gt_csv, index=False, encoding="utf-8-sig")

    # ============================================================
    # ✅ Step 4: Display Summary
    # ============================================================
    print(f"\n✅ {len(results)} Spanish audio files generated and saved to: {output_folder}")
    print(f"📄 Summary saved to: {summary_csv}")
    print(f"📄 Ground truth saved to: {gt_csv}")

    return pd.DataFrame(results)


In [9]:
# ============================================================
# 1️⃣ Mount Google Drive
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

# ============================================================
# 2️⃣ Define your folder path in Drive
# ============================================================
AUDIO_INPUT_FOLDER = "/content/drive/MyDrive/health-tequity-case/Input_Audio_Files"

# ============================================================
# 3️⃣ Define your English questions
# ============================================================
english_questions = [
    "What are my systolic and diastolic blood pressures today?",
    "What were my blood pressure values over the last week?",
    "What is the trend of my blood pressure values?",
    "What are the normal ranges for a person like me?",
    "What was my blood pressure on October 10th?",
    "On which day did my blood pressure exceed normal levels?"
    "Compare my average blood pressure in the first week and last week of this month."
    "What was my lowest diastolic blood pressure this month?"


]

# ============================================================
# 4️⃣ Generate Spanish audio files and CSVs
# ============================================================
audio_summary_df = generate_spanish_audio_from_english(
    english_questions,
    output_folder=AUDIO_INPUT_FOLDER
)

display(audio_summary_df)


KeyboardInterrupt: 